# Information Retrieval and Web Analytics

# Part 1: Text Processing


In [1]:
# mount google drive if using google collab, else skip
# we are not using it because it is more comfortable to use jupyter lab

try:
    from google.colab import drive
    drive.mount('/content/drive')
    
except ModuleNotFoundError:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# required imports for the notebook

import json
import csv
import re

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


In [4]:
import pandas as pd
df = pd.read_json('drive/MyDrive/data/tw_hurricane_data.json',lines=True)

In [5]:
new = df[['id','full_text', 'user','created_at','entities', 'favorite_count', 'retweet_count']].copy()

In [6]:
url=[]
has=[]
user=[]
for ele in range(len(new)):
  has.append(new['entities'][ele]['hashtags'][0]["text"])
  user.append(new['user'][ele]['name'])
  try:
    url.append(new['entities'][ele]['media'][0]["expanded_url"])
  except: 
    url.append('')
new['url'] = url
new['hashtags'] = has
new['user'] = user

In [7]:
def build_terms(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    line = line.lower()
    line = line.split()  # Tokenize the text to get a list of terms
    line = [x for x in line if x not in stop_words]  # eliminate the stopwords
    line = [stemmer.stem(word) for word in line] # perform stemming (HINT: use List Comprehension)
    return line

In [9]:
tmp = build_terms(new['full_text'][0])
print(tmp)
tmp = [word for word in tmp if word.startswith('#')==False]
print(tmp)
' '.join(tmp)

['keep', 'spin', 'us', '7', 'pm…go', 'away', 'already.', '#hurricaneian', 'https://t.co/vrotxns9rz']
['keep', 'spin', 'us', '7', 'pm…go', 'away', 'already.', 'https://t.co/vrotxns9rz']


'keep spin us 7 pm…go away already. https://t.co/vrotxns9rz'

In [10]:
for ele in range(len(new)):
  tmp = build_terms(new['full_text'][ele])
  tmp = [word for word in tmp if word.startswith('#')==False]
  tmp = [word for word in tmp if word.startswith('@')==False]
  tmp = [word for word in tmp if word.startswith('http')==False]
  new['full_text'][ele] = ' '.join(tmp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
new.drop(['entities'], axis=1)

,id,full_text,user,created_at,favorite_count,retweet_count,url,hashtags
0,doc_1,keep spin us 7 pm…go away already.,Suz👻,2022-09-30 18:39:08+00:00,0,0,https://twitter.com/suzjdean/status/1575918182...,HurricaneIan
1,doc_2,heart go affect wish everyon road current brav...,Lytx,2022-09-30 18:39:01+00:00,0,0,,HurricaneIan
2,doc_3,kissimme neighborhood michigan ave.,Christopher Heath,2022-09-30 18:38:58+00:00,0,0,https://twitter.com/CHeathWFTV/status/15759181...,HurricaneIan
3,doc_4,one tree backyard scare poltergeist tree it’ s...,alex ✨,2022-09-30 18:38:57+00:00,0,0,,scwx
4,doc_5,pray everyon affect associ winknews. sympathi ...,Tess 💋,2022-09-30 18:38:53+00:00,0,0,,HurricaneIan
...,...,...,...,...,...,...,...,...
3995,doc_3996,"cfrd, , carrboro public works, servic place st...",Carrboro Fire-Rescue,2022-09-30 14:33:06+00:00,2,0,,CarrboroSafe
3996,doc_3997,isn’t list website? widespread flood &amp; bc ...,BaconBitsNews,2022-09-30 14:33:01+00:00,0,0,,Kissimmee
3997,doc_3998,realli flood flute,@jganyfl,2022-09-30 14:32:57+00:00,16,8,https://twitter.com/jganyfl1/status/1575856228...,HurricaneIan
3998,doc_3999,damag area punta gorda...a thread. tropic gulf...,CJ Haddad,2022-09-30 14:32:56+00:00,2,1,https://twitter.com/haddad_cj/status/157585622...,HurricaneIan


In [ ]:
# get dictionary to map tweet ids to doc ids
# we know the ids file is a list of [doc_id \t tweet_id]
with open("drive/MyDrive/data/tweet_document_ids_map.csv", "r") as id_file:
    ids = csv.reader(id_file, delimiter="\t")
    dict_ids = {id_to_id[1]: id_to_id[0] for id_to_id in list(ids)}

In [13]:
# map tweet ids with doc ids
for ele in range(len(new)):
    tweet_id = str(new["id"][ele])
    new["id"][ele] = dict_ids[tweet_id]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
